In [1]:
mkdir qlora-replication
cd qlora-replication
mkdir logs
ls

logs


In [4]:
pip install -q -U -r https://raw.githubusercontent.com/artidoro/qlora/main/requirements.txt
pip install -q protobuf==3.19
pip install -q scipy

In [10]:
mv /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so \
   /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so.bak

In [11]:
cp /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so \
   /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so

In [12]:
mkdir /usr/local/cuda
mkdir /usr/local/cuda/lib64
ln -sf /opt/conda/lib/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so

In [20]:
mkdir qlora-replication
mkdir qlora-replication/qlora

In [21]:
git clone https://github.com/liguodongiot/llm-action.git
cp llm-action/qlora/* qlora-replication/qlora/
rm -rf llm-action

Cloning into 'llm-action'...
remote: Enumerating objects: 761, done.        
remote: Counting objects: 100% (123/123), done.        
remote: Compressing objects: 100% (119/119), done.        
remote: Total 761 (delta 65), reused 0 (delta 0), pack-reused 638        
Receiving objects: 100% (761/761), 660.05 KiB | 6.29 MiB/s, done.
Resolving deltas: 100% (384/384), done.


In [2]:
pip3 install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 12.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 38.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 68.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 52.2 MB/s eta 0:00:00



In [ ]:
model_size="13b"

cd /workspace/qlora-replication/qlora
torchrun --nproc_per_node=1 qlora.py \
    --model_name_or_path "huggyllama/llama-${model_size}" \
    --output_dir "output/llama-${model_size}-qlora" \
    --per_device_train_batch_size 1 \
    --max_steps 10000 \
    --save_total_limit 2


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib'), PosixPath('/usr/local/nvidia/lib64')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following di

In [10]:
pip install -q -r https://raw.githubusercontent.com/hiyouga/LLaMA-Efficient-Tuning/main/requirements.txt

In [5]:
cd /workspace/qlora-replication
git clone https://github.com/hiyouga/LLaMA-Efficient-Tuning.git

Cloning into 'LLaMA-Efficient-Tuning'...
remote: Enumerating objects: 620, done.        
remote: Counting objects: 100% (366/366), done.        
remote: Compressing objects: 100% (147/147), done.        
remote: Total 620 (delta 287), reused 269 (delta 215), pack-reused 254        
Receiving objects: 100% (620/620), 70.83 MiB | 16.52 MiB/s, done.
Resolving deltas: 100% (430/430), done.
Updating files: 100% (46/46), done.


In [1]:
cd /workspace/qlora-replication/LLaMA-Efficient-Tuning

In [2]:
model_size="13b"
WANDB_DISABLED=true
# accelerate config # configure the environment
accelerate launch src/train_sft.py \
    --model_name_or_path "huggyllama/llama-${model_size}" \
    --do_train \
    --dataset alpaca_en \
    --finetuning_type lora \
    --output_dir "output/llama-${model_size}-qlora-let" \
    --quantization_bit 4 \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/usr/local/nvidia/lib64'), PosixPath('/usr/local/nvidia/lib')}
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: /usr/local/nvidia/lib:/usr/local/nvidia/lib64 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cuda_setup/main.py:149: UserWarning: WARNING: The following di

In [3]:
mkdir /workspace/qlora-replication/logs

In [4]:
model_size="13b"
WANDB_DISABLED=true
nohup python src/train_sft.py \
    --model_name_or_path "huggyllama/llama-${model_size}" \
    --do_train \
    --dataset alpaca_en \
    --finetuning_type lora \
    --output_dir "output/llama-${model_size}-qlora-let" \
    --quantization_bit 4 \
    --overwrite_cache \
    --per_device_train_batch_size 4 \
    --gradient_accumulation_steps 4 \
    --lr_scheduler_type cosine \
    --logging_steps 10 \
    --save_steps 1000 \
    --learning_rate 5e-5 \
    --num_train_epochs 3.0 \
    --plot_loss \
    --fp16 > "/workspace/qlora-replication/logs/lama-${model_size}-qlora-let" 2>&1

In [29]:
jobs